In [4]:
import pandas as pd
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

**Data for per game stats for players from the 2024 season.**

In [13]:
player_stats_url = "https://www.basketball-reference.com/leagues/NBA_2024_per_game.html"
data = requests.get(player_stats_url)
time.sleep(15)
with open("player_stats/2024.html", "w+") as f:
    f.write(data.text)

In [14]:
driver = webdriver.Chrome(
    service=Service(executable_path="C:/Users/vuyyu/Dropbox/PC/Desktop/Program_Installers/chromedriver-win64/chromedriver-win64/chromedriver.exe")
    )

In [15]:
url = player_stats_url

driver.get(url)
driver.execute_script("window.scrollTo(1,10000)")
time.sleep(2)

with open("player_stats/2024.html", "w+") as f:
    f.write(driver.page_source)

In [16]:
dfs = []
with open("player_stats/2024.html") as f:
    page = f.read()

soup = BeautifulSoup(page, 'html.parser')
soup.find('tr', class_="thead").decompose()
player_table = soup.find_all(id="per_game_stats")[0]
player_df = pd.read_html(str(player_table))[0]
player_df["Year"] = 2024
dfs.append(player_df)

In [18]:
players_2024 = pd.concat(dfs)
players_2024.to_csv("2024_players.csv")

**Data for advanced stats for players from the 2024 season.**

In [26]:
advanced_stats_url = "https://www.basketball-reference.com/leagues/NBA_2024_advanced.html"

data = requests.get(advanced_stats_url)
time.sleep(15)

with open("advanced_stats/2024.html", "w+") as f:
    f.write(data.text)

In [27]:
url = advanced_stats_url

driver.get(url)
driver.execute_script("window.scrollTo(1,10000)")
time.sleep(2)

with open("advanced_stats/2024.html", "w+") as f:
    f.write(driver.page_source)

In [28]:
dfs = []
with open("advanced_stats/2024.html") as f:
    page = f.read()

soup = BeautifulSoup(page, 'html.parser')
soup.find('tr', class_="thead").decompose()
player_table = soup.find_all(id="advanced_stats")[0]
player_df = pd.read_html(str(player_table))[0]
player_df["Year"] = 2024
dfs.append(player_df)

In [29]:
players_advanced = pd.concat(dfs)
players_advanced.head()

,Rk,Player,Pos,Age,Tm,G,MP,PER,TS%,3PAr,...,OWS,DWS,WS,WS/48,Unnamed: 24,OBPM,DBPM,BPM,VORP,Year
0,1,Precious Achiuwa,PF-C,24,TOT,63,1432,14.9,.547,.215,...,1.1,2.0,3.2,.106,NaN,-1.5,0.5,-1.0,0.4,2024
1,1,Precious Achiuwa,C,24,TOR,25,437,14.9,.512,.276,...,0.0,0.5,0.5,.052,NaN,-1.5,-0.1,-1.6,0.0,2024
2,1,Precious Achiuwa,PF,24,NYK,38,995,14.8,.570,.174,...,1.1,1.6,2.7,.129,NaN,-1.6,0.8,-0.8,0.3,2024
3,2,Bam Adebayo,C,26,MIA,60,2060,19.9,.572,.025,...,2.3,3.6,6.0,.139,NaN,0.7,1.6,2.3,2.2,2024
4,3,Ochai Agbaji,SG,23,TOT,70,1421,7.9,.504,.516,...,-0.4,0.6,0.3,.009,NaN,-3.3,-0.9,-4.2,-0.8,2024


In [30]:
del players_advanced["Unnamed: 19"]
del players_advanced["Unnamed: 24"]

In [31]:
players_advanced.to_csv("2024_advanced_stats.csv")

**Cleaning and combining advanced stats and regular stats.**

In [5]:
players = pd.read_csv("2024_players.csv")
del players["Rk"]
del players["Unnamed: 0"]

In [6]:
advanced_stats = pd.read_csv("2024_advanced_stats.csv")
del advanced_stats["Unnamed: 0"]

In [7]:
players["Player"] = players["Player"].str.replace("*","",regex=False)
advanced_stats["Player"] = advanced_stats["Player"].str.replace("*","",regex=False)

In [8]:
def single_row(df):
    if df.shape[0] == 1:
        return df
    else:
        row = df[df["Tm"] == "TOT"]
        row["Tm"] = df.iloc[-1,:]["Tm"]
        return row
advanced_stats = advanced_stats.groupby(["Player", "Year"]).apply(single_row)
players = players.groupby(["Player", "Year"]).apply(single_row)

In [9]:
advanced_stats.index = advanced_stats.index.droplevel()
advanced_stats.index = advanced_stats.index.droplevel()

In [10]:
players.index = players.index.droplevel()
players.index = players.index.droplevel()

In [11]:
advanced_stats = advanced_stats[['Player','Year','PER', 'TS%',
       '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%',
       'USG%', 'OWS', 'DWS', 'WS', 'WS/48', 'OBPM', 'DBPM', 'BPM', 'VORP']]

In [12]:
advanced_stats

,Player,Year,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,...,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP
243,A.J. Green,2024,11.9,.636,.861,.097,1.9,10.2,6.1,7.4,...,4.4,16.0,0.8,0.2,1.1,.105,0.8,-1.9,-1.1,0.1
365,A.J. Lawson,2024,12.8,.552,.447,.223,5.4,9.9,7.6,7.9,...,8.8,19.7,0.1,0.2,0.3,.062,-2.2,-1.2,-3.5,-0.1
250,AJ Griffin,2024,1.8,.403,.733,.044,1.6,10.2,5.7,3.8,...,11.6,16.3,-0.3,0.0,-0.3,-0.097,-5.4,-3.3,-8.7,-0.2
239,Aaron Gordon,2024,16.6,.601,.183,.361,8.8,14.0,11.5,14.4,...,11.1,17.9,3.7,2.3,6.0,.140,0.9,0.0,0.9,1.5
288,Aaron Holiday,2024,11.3,.580,.531,.142,1.6,8.7,5.2,15.4,...,11.0,16.5,1.0,1.2,2.2,.094,-1.8,0.4,-1.3,0.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136,Zach Collins,2024,13.6,.562,.302,.259,8.1,17.4,12.7,18.5,...,16.7,22.1,0.1,1.1,1.2,.043,-2.5,-0.4,-2.8,-0.3
364,Zach LaVine,2024,15.1,.578,.449,.274,1.0,15.9,8.3,17.0,...,11.0,23.8,0.8,0.7,1.4,.078,0.8,-0.7,0.1,0.5
487,Zeke Nnaji,2024,12.4,.522,.154,.434,12.7,11.2,11.9,7.2,...,14.3,15.8,0.3,0.7,1.0,.091,-3.9,0.1,-3.8,-0.2
716,Ziaire Williams,2024,9.3,.511,.500,.198,3.6,15.8,9.5,11.2,...,13.8,19.8,-0.9,1.2,0.2,.010,-4.1,-0.4,-4.5,-0.6


In [13]:
stats = players.merge(advanced_stats, how="outer", on=["Player", "Year"])
stats

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP
0,A.J. Green,SG,24,MIL,47,0,10.3,1.5,3.5,.436,...,4.4,16.0,0.8,0.2,1.1,.105,0.8,-1.9,-1.1,0.1
1,A.J. Lawson,SG,23,DAL,33,0,7.4,1.3,2.8,.468,...,8.8,19.7,0.1,0.2,0.3,.062,-2.2,-1.2,-3.5,-0.1
2,AJ Griffin,SF,20,ATL,18,0,7.3,0.7,2.5,.289,...,11.6,16.3,-0.3,0.0,-0.3,-0.097,-5.4,-3.3,-8.7,-0.2
3,Aaron Gordon,PF,28,DEN,65,65,31.5,5.5,9.9,.553,...,11.1,17.9,3.7,2.3,6.0,.140,0.9,0.0,0.9,1.5
4,Aaron Holiday,PG,27,HOU,67,1,16.4,2.4,5.3,.447,...,11.0,16.5,1.0,1.2,2.2,.094,-1.8,0.4,-1.3,0.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
557,Zach Collins,C,26,SAS,58,26,22.6,4.3,8.9,.482,...,16.7,22.1,0.1,1.1,1.2,.043,-2.5,-0.4,-2.8,-0.3
558,Zach LaVine,SG,28,CHI,25,23,34.9,6.8,15.0,.452,...,11.0,23.8,0.8,0.7,1.4,.078,0.8,-0.7,0.1,0.5
559,Zeke Nnaji,PF,23,DEN,53,0,9.8,1.2,2.6,.449,...,14.3,15.8,0.3,0.7,1.0,.091,-3.9,0.1,-3.8,-0.2
560,Ziaire Williams,SF,22,MEM,51,15,20.4,2.9,7.4,.397,...,13.8,19.8,-0.9,1.2,0.2,.010,-4.1,-0.4,-4.5,-0.6


**Team standing (3/25/2024)**

In [72]:
team_stats_url = "https://www.basketball-reference.com/leagues/NBA_2024_standings.html"

In [5]:
url = team_stats_url

data = requests.get(url)
time.sleep(15)

with open("team_standings/2024.html", "w+") as f:
    f.write(data.text)

In [6]:
dfs = []
with open("team_standings/2024.html") as f:
    page = f.read()

soup = BeautifulSoup(page, 'html.parser')
soup.find('tr', class_="thead").decompose()
e_table = soup.find_all(id="divs_standings_E")[0]
e_df = pd.read_html(str(e_table))[0]
e_df["Year"] = 2024
e_df["Team"] = e_df["Eastern Conference"]
del e_df["Eastern Conference"]
dfs.append(e_df)

w_table = soup.find_all(id="divs_standings_W")[0]
w_df = pd.read_html(str(w_table))[0]
w_df["Year"] = 2024
w_df["Team"] = w_df["Western Conference"]
del w_df["Western Conference"]
dfs.append(w_df)

In [7]:
team_standings = pd.concat(dfs)

In [8]:
team_standings.to_csv("2024_team_standings.csv")

**Adding team standings to player dataset**

In [14]:
teams = pd.read_csv("2024_team_standings.csv")
teams = teams[~teams["W"].str.contains("Division")]

In [15]:
nicknames = {}

with open("nicknames.txt") as f:
    lines = f.readlines()
    for line in lines[1:]:
        abbrev, name = line.replace("\n", "").split(",")
        nicknames[abbrev] = name

In [16]:
stats["Team"] = stats["Tm"].map(nicknames)

In [17]:
stats = stats.merge(teams, how = "outer", on = ["Team", "Year"])

In [18]:
del stats["Unnamed: 0"]

In [19]:
stats = stats.apply(pd.to_numeric, errors="ignore")

In [20]:
stats.dtypes

Player     object
Pos        object
Age         int64
Tm         object
G           int64
GS          int64
MP        float64
FG        float64
FGA       float64
FG%       float64
3P        float64
3PA       float64
3P%       float64
2P        float64
2PA       float64
2P%       float64
eFG%      float64
FT        float64
FTA       float64
FT%       float64
ORB       float64
DRB       float64
TRB       float64
AST       float64
STL       float64
BLK       float64
TOV       float64
PF        float64
PTS       float64
Year        int64
PER       float64
TS%       float64
3PAr      float64
FTr       float64
ORB%      float64
DRB%      float64
TRB%      float64
AST%      float64
STL%      float64
BLK%      float64
TOV%      float64
USG%      float64
OWS       float64
DWS       float64
WS        float64
WS/48     float64
OBPM      float64
DBPM      float64
BPM       float64
VORP      float64
Team       object
W           int64
L           int64
W/L%      float64
GB         object
PS/G      

In [21]:
stats["GB"].unique()

array(['—', '2.5', '10.5', '8.5', '6.5', '0.5', '12.0', '14.5', '24.5',
       '5.5', '3.0', '8.0', '31.0', '1.0', '29.0', '34.0', '21.0', '18.0',
       '20.0'], dtype=object)

In [22]:
stats["GB"] = stats["GB"].str.replace("—","0")

In [23]:
stats["GB"] = pd.to_numeric(stats["GB"])
stats.dtypes

Player     object
Pos        object
Age         int64
Tm         object
G           int64
GS          int64
MP        float64
FG        float64
FGA       float64
FG%       float64
3P        float64
3PA       float64
3P%       float64
2P        float64
2PA       float64
2P%       float64
eFG%      float64
FT        float64
FTA       float64
FT%       float64
ORB       float64
DRB       float64
TRB       float64
AST       float64
STL       float64
BLK       float64
TOV       float64
PF        float64
PTS       float64
Year        int64
PER       float64
TS%       float64
3PAr      float64
FTr       float64
ORB%      float64
DRB%      float64
TRB%      float64
AST%      float64
STL%      float64
BLK%      float64
TOV%      float64
USG%      float64
OWS       float64
DWS       float64
WS        float64
WS/48     float64
OBPM      float64
DBPM      float64
BPM       float64
VORP      float64
Team       object
W           int64
L           int64
W/L%      float64
GB        float64
PS/G      

In [24]:
stats.to_csv("2024_player_mvp_stats.csv")